In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("data.csv") 

In [3]:
df.head()

,a_qualifier,b_qualifier,event_name,event_id,event_slug,no_teams,prize,is_qualifier,type,country,...,at,bct,act,bt,ar,br,afk,bfk,acw,bcw
0,False,False,EMF CS:GO World Invitational 2019,4730.0,emf-csgo-world-invitational-2019,2.0,64100.0,False,Intl. LAN,Hong Kong,...,7.0,0.0,9.0,6.0,1.42,0.68,15.0,7.0,2.0,0.0
1,False,False,EMF CS:GO World Invitational 2019,4730.0,emf-csgo-world-invitational-2019,2.0,64100.0,False,Intl. LAN,Hong Kong,...,5.0,7.0,6.0,9.0,0.87,1.22,14.0,13.0,1.0,3.0
2,False,False,EMF CS:GO World Invitational 2019,4730.0,emf-csgo-world-invitational-2019,2.0,64100.0,False,Intl. LAN,Hong Kong,...,6.0,9.0,3.0,7.0,1.01,1.18,9.0,16.0,2.0,3.0
3,False,False,EMF CS:GO World Invitational 2019,4730.0,emf-csgo-world-invitational-2019,2.0,64100.0,False,Intl. LAN,Hong Kong,...,2.0,9.0,8.0,7.0,0.82,1.26,7.0,19.0,1.0,0.0
4,False,False,IEM Chicago 2019,4574.0,iem-chicago-2019,8.0,250000.0,False,Intl. LAN,United States,...,1.0,2.0,1.0,14.0,0.62,1.64,8.0,10.0,0.0,3.0


In [ ]:
df.drop(columns=['event_name','event_slug','from','to','matchid','seriesid','eventid','url',"event_id",'seriestype','country'], inplace=True)

In [ ]:
df.prize.mean()

In [ ]:
df.prize.fillna(df.prize.mean(), inplace=True)

In [ ]:
df.dropna(subset=['a'],inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['acw','bcw','afk','bfk','ar','br'], inplace=True)

In [ ]:
df = pd.get_dummies(df,columns=['type','a','b','map'],prefix=['type','a','b','map'])

In [ ]:
df['a_win'] = df['at'] + df.act > df.bt + df.bct

In [ ]:
test = df.drop(columns=['af5r','bf5r','atpis','actpis','btpis','bctpis','at','act','bt','bct'])

In [ ]:
from sklearn.model_selection import train_test_split #for split the data
from sklearn.metrics import accuracy_score  #for accuracy_score
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.metrics import confusion_matrix #for confusion matrix
all_features = test.drop("a_win",axis=1)
Targeted_feature = test["a_win"]
X_train,X_test,y_train,y_test = train_test_split(all_features,Targeted_feature,test_size=0.3,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
# Random Forests
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(criterion='gini', n_estimators=700,
                             min_samples_split=10,min_samples_leaf=1,
                             max_features='auto',oob_score=True,
                             random_state=1,n_jobs=-1)
model.fit(X_train,y_train)
prediction_rm=model.predict(X_test)
print('--------------The Accuracy of the model----------------------------')
print('The accuracy of the Random Forest Classifier is',round(accuracy_score(prediction_rm,y_test)*100,2))
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
result_rm=cross_val_score(model,all_features,Targeted_feature,cv=10,scoring='accuracy')
print('The cross validated score for Random Forest Classifier is:',round(result_rm.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap="summer")
plt.title('Confusion_matrix', y=1.05, size=15)